In [ ]:

# Install necessary libraries
!pip install transformers datasets torch tqdm

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from datasets import Dataset

dataset_path = "/content/drive/My Drive/listops/basic_train.tsv"  # Update with your file path
df = pd.read_csv(dataset_path, sep='\t')
df_base=df.copy()

dataset = Dataset.from_pandas(df)

print(dataset)
print(dataset.features)


Dataset({
    features: ['Source', 'Target'],
    num_rows: 96000
})
{'Source': Value(dtype='string', id=None), 'Target': Value(dtype='int64', id=None)}


In [ ]:
def clean_sequence(sequence):
    """
    Cleans a sequence by removing excessive spaces or tabs between important characters.
    Args:
        sequence (str): The input sequence to clean.
    Returns:
        str: The cleaned sequence.
    """
    cleaned_sequence = ''.join(sequence.split())
    return cleaned_sequence




In [ ]:
print(len(df.head()['Source'][0]))



8538


In [ ]:
df['Source'] = df['Source'].apply(clean_sequence)

print(len(df.head()['Source'][0]))

4536


In [ ]:
print((df.head()['Source'][0]))
print((df_base.head()['Source'][0]))


((((((((((([MIN3)7)(((((((((([MAX((([MIN9)5)]))4)0)4)1)4)9)8)8)]))8)2)((((([MAX9)7)((((([MAX4)9)((((([SM6)7)6)((((((((([SM8)2)7)(((((([MAX6)5)9)6)2)]))((((((((((([MAX7)0)7)0)((((((([MED((((((([MAX4)1)6)6)((([MAX3)1)]))9)]))8)5)7)8)0)]))9)8)2)0)((((([MAX2)((((((((((([SM0)(((((([MAX5)6)0)9)6)]))((([MED0)5)]))((((((((((([MIN1)0)3)0)8)1)4)5)9)6)]))7)(((((((([MAX2)5)8)9)8)7)1)]))6)((([MAX0)5)]))6)6)]))9)6)]))]))0)9)6)]))]))4)]))(((((((((([SM2)(((((((([MED(((((([MIN(((((((((([MED1)7)1)((((((((((([MED6)2)3)2)1)7)6)5)(((((((((([SM4)(((((((([MAX9)1)5)4)7)4)8)]))3)3)(((((((([SM6)9)9)0)7)6)9)]))3)9)2)5)]))(((([MAX0)0)2)]))]))8)2)6)((((((((([MED1)2)((((([MAX8)7)2)(((((((((([MAX7)4)6)9)6)8)6)4)8)]))]))0)((((((((((([SM1)7)9)9)8)(((((((([MAX8)6)0)7)5)4)8)]))1)0)7)0)]))4)1)1)]))8)]))7)5)8)9)]))3)(((((((((([MIN(((((((((([MAX((((((((([SM6)8)4)2)8)9)8)(((((((((([MAX8)0)3)3)(((((((((([MED4)6)2)9)7)2)6)4)1)]))((([SM4)2)]))0)1)6)]))]))9)3)2)2)2)(((((((([MED(((((([MED(((([MAX6)7)4)]))8)(((((([MIN0)9)0)6)8)])

In [ ]:

#---- chatgpt code
def preprocess_data_for_inference(examples):
    return tokenizer(
        examples['Source'],
        padding='max_length',
        max_length=4096
    
    )

encoded_dataset = dataset.map(preprocess_data_for_inference, batched=True)
#---- chatgpt code


In [ ]:
label_mapping = {label: idx for idx, label in enumerate(sorted(df['Target'].unique()))}
df['Target'] = df['Target'].map(label_mapping)
print(encoded_dataset.features)


NameError: name 'encoded_dataset' is not defined

In [ ]:
from transformers import LongformerForSequenceClassification


num_labels = len(label_mapping)
model = LongformerForSequenceClassification.from_pretrained(
    'allenai/longformer-base-4096',
    num_labels=num_labels
)



Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments
#---- chatgpt code
training_args = TrainingArguments(
    output_dir='./results',       
    evaluation_strategy="epoch", 
    learning_rate=5e-5,         
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",        
    save_total_limit=2,          
    load_best_model_at_end=True, 
    logging_dir='./logs',       
    logging_steps=10,
    fp16=True,
    report_to="none"
)
#---- chatgpt code


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")



In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-19-a5113f9b8f72>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,2.225000,2.230801
2,2.199600,2.222827
3,2.297600,2.223965


TrainOutput(global_step=600, training_loss=2.245191650390625, metrics={'train_runtime': 4970.7246, 'train_samples_per_second': 0.966, 'train_steps_per_second': 0.121, 'total_flos': 1.26122766630912e+16, 'train_loss': 2.245191650390625, 'epoch': 3.0})

In [ ]:

metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 2.222827196121216, 'eval_runtime': 211.0878, 'eval_samples_per_second': 1.895, 'eval_steps_per_second': 0.947, 'epoch': 3.0}


In [ ]:

model.save_pretrained("/content/drive/My Drive/longformer_listops_model")
tokenizer.save_pretrained("/content/drive/My Drive/longformer_listops_model")


('/content/drive/My Drive/longformer_listops_model/tokenizer_config.json',
 '/content/drive/My Drive/longformer_listops_model/special_tokens_map.json',
 '/content/drive/My Drive/longformer_listops_model/vocab.json',
 '/content/drive/My Drive/longformer_listops_model/merges.txt',
 '/content/drive/My Drive/longformer_listops_model/added_tokens.json')

                                              Source  Target
0  ( ( ( ( ( ( ( ( ( ( ( [MIN 3 ) 7 ) ( ( ( ( ( (...       1
1  ( ( ( ( ( [SM 3 ) 5 ) ( ( ( ( ( ( [MAX 4 ) ( (...       5
2  ( ( ( ( ( ( ( [MED ( ( ( [MIN ( ( ( ( ( ( ( ( ...       5
3  ( ( ( ( ( ( ( ( ( ( ( [MAX ( ( ( ( ( ( ( ( [ME...       9
4  ( ( ( ( ( ( ( ( ( ( ( [SM ( ( ( [SM 4 ) 8 ) ] ...       5


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import LongformerForSequenceClassification, LongformerTokenizer
import torch

model_path = "/content/drive/My Drive/longformer_listops_model"
model = LongformerForSequenceClassification.from_pretrained(model_path).to('cuda') 
tokenizer = LongformerTokenizer.from_pretrained(model_path)

dataset_path = "/content/drive/My Drive/listops/basic_train.tsv"  
df = pd.read_csv(dataset_path, sep='\t') 
print(df.head())

In [ ]:

df_truncated = df.head(10) 



dataset = Dataset.from_pandas(df_truncated)

def preprocess_data_for_inference(examples):
    return tokenizer(
        examples['Source'],
        truncation=True,
        padding='max_length',
        max_length=4096
    )

processed_dataset = dataset.map(preprocess_data_for_inference, batched=True)

processed_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
print(processed_dataset)

data_loader = torch.utils.data.DataLoader(
    processed_dataset,
    batch_size=1,
)

model.eval()

all_predictions = []

for batch in data_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        print(logits)
        predictions = torch.argmax(logits, dim=-1)

    all_predictions.extend(predictions.cpu().numpy())

for i in range(len(all_predictions)):
    print(f"Ground Truth: {df_truncated['Target'].iloc[i]}, Prediction: {all_predictions[i]}")


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['Source', 'Target', 'input_ids', 'attention_mask'],
    num_rows: 10
})
tensor([[ 0.6879, -0.0994, -0.3267, -0.3459,  0.0540, -0.1160, -0.1382, -0.1948,
         -0.2868,  0.6062]], device='cuda:0')
tensor([[ 0.6879, -0.0994, -0.3267, -0.3459,  0.0540, -0.1160, -0.1382, -0.1948,
         -0.2868,  0.6061]], device='cuda:0')
tensor([[ 0.6879, -0.0994, -0.3267, -0.3459,  0.0540, -0.1160, -0.1382, -0.1948,
         -0.2868,  0.6061]], device='cuda:0')
tensor([[ 0.6879, -0.0994, -0.3267, -0.3459,  0.0540, -0.1160, -0.1382, -0.1948,
         -0.2868,  0.6062]], device='cuda:0')
tensor([[ 0.6879, -0.0994, -0.3267, -0.3459,  0.0540, -0.1160, -0.1382, -0.1948,
         -0.2868,  0.6062]], device='cuda:0')
tensor([[ 0.6879, -0.0994, -0.3267, -0.3459,  0.0540, -0.1160, -0.1382, -0.1948,
         -0.2868,  0.6061]], device='cuda:0')
tensor([[ 0.6879, -0.0994, -0.3267, -0.3459,  0.0540, -0.1160, -0.1382, -0.1948,
         -0.2868,  0.6061]], device='cuda:0')
tensor([[ 0.68